In [51]:
import requests
from bs4 import BeautifulSoup

url = "https://m.coa.gov.tw/FarmProduceOriginPrice/Index#"
res = requests.get(url)
soup = BeautifulSoup(res.text)
print(soup.title.string)


        農產品產地價格查詢 | 田邊好幫手
    


In [48]:
#找出所有蔬菜水果的種類
pd_name = soup.find_all("select", class_="custom-select")
pd_name_list = []
len(pd_name[1].find_all("option"))
for i in pd_name[1].find_all("option"):
    pd_name_list.append(i.text) 
#移除第一個選項 - 請選擇
pd_name_list.pop(0)

num = 0
#花的單位都不是(元/公斤)
#for list，要移除內容物時，要複製出另一個list
for i in pd_name_list:
    #print(i)
    if "(元/公斤)" not in i and "(元/公斤 )" not in i and "(元/ 公斤)" not in i:
        print("第一次："+i)
        pd_name_list.remove(i)
        num+=1
print(num)
print(pd_name_list)

for i in pd_name_list:
    #print(i)
    if "(元/公斤)" not in i and "(元/公斤 )" not in i and "(元/ 公斤)" not in i:
        print("第二次："+i)
        pd_name_list.remove(i)
print(pd_name_list)

第一次：大理花(元/支)
第一次：天堂鳥(元/支)
第一次：火鶴(元/支)
第一次：卡斯比亞(元/把)
第一次：夜來香(元/支)
第一次：玫瑰(元/支)
第一次：非洲菊(元/支)
第一次：香水百合(元/支)
第一次：香水百合(黃)(元/支)
第一次：香石竹(康乃馨)大輪(元/支)
第一次：唐菖蒲(劍蘭)(元/支)
第一次：菊(多花小菊)(元/支)
第一次：聖誕紅(元/盆)
第一次：葵百合(紅)(元/支)
第一次：滿天星(元/ 把)
第一次：檳榔(元/ 粒)
16
第二次：文心蘭(元/支)
第二次：火鶴(紅)(元/支)
第二次：玫瑰(黛安娜)(元/支)
第二次：香水百合(白)(元/支)
第二次：香石竹(康乃馨)中康(元/支)
第二次：姬百合(橙)(元/支)
第二次：菊花(大菊)(元/支)
['大豆(元/公斤)', '大豆(黑豆)(元/公斤)', '大芥菜(鮮食)(元/公斤)', '小白菜(元/公斤)', '小番茄(玉女)(元/公斤)', '小蕃茄(聖女)(元/公斤)', '山藥(元/公斤)', '五爪蘋果(元/公斤)', '文旦(元/公斤)', '木瓜(元/公斤)', '木耳(黑色溼)(元/公斤)', '毛豆(帶莢)(元/公斤)', '水蜜桃(元/公斤)', '牛心柿(元/公斤)', '牛蒡(元/公斤)', '牛蕃茄(元/公斤)', '冬瓜(元/公斤)', '加工麻竹筍(元/公斤)', '巨峰葡萄(元/公斤)', '本島萵苣(元/公斤)', '玉米筍(元/公斤)', '甘藍(元/公斤)', '甘藍芽(元/公斤)', '甘藷(元/公斤)', '生薑(老薑粉薑)(元/公斤)', '生薑(嫩薑)(元/公斤)', '白玉蘿蔔(元/公斤)', '白柚(元/公斤)', '白蘆筍(元/公斤)', '百香果(元/公斤)', '西瓜(大粒)(元/公斤)', '西瓜(小粒)(元/公斤)', '西瓜(無子)(元/公斤)', '李(玉李)(元/公斤)', '李(紅肉)(元/公斤)', '杏鮑菇(元/公斤)', '芋頭(元/公斤)', '芒果(在來)(元/公斤)', '芒果(金煌)(元/公斤)', '芒果(愛文)(元/公斤)', '咖啡(水洗生豆)(元/公斤)', '咖啡(生豆)(元/公斤)', '咖啡(櫻桃豆)(元/公斤)', '幸水梨(元/公斤)', '枇杷(元/公斤)', '芝麻(黑省產

In [215]:
#動態爬蟲
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import os

In [216]:
#開啟瀏覽器
driver = webdriver.Chrome("/Users/ben/Documents/chromedriver")
driver.get("https://m.coa.gov.tw/FarmProduceOriginPrice/Index#")

<ipython-input-216-9a048df2aff1>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/ben/Documents/chromedriver")


In [217]:
select = Select(driver.find_element_by_name("CropName"))
search = driver.find_element_by_id("btn-search")
data = {
    "作物":[],
    "月份":[],
    "價格":[]
}

for name in pd_name_list:
    select.select_by_value(name)
    search.click()
    time.sleep(10)
    #抓取內容
    inner_content = driver.find_elements_by_css_selector("tbody>tr>td")
    inner_list = []
    #在底下的for loop中time sleep
    for i in inner_content:
        if i.text == "": #清除空字串
            time.sleep(0.25)
            continue
        else:
            inner_list.append(i.text)
            time.sleep(0.25)
    print(inner_list)
    
    #找出有2022年資料的農作物
    if inner_list[1] == "2022年":
        inner_list = inner_list[5:] #移除2022年的資料
    if inner_list[13] == inner_list[0]: #大芥菜鮮食，2021年12月沒資料
        inner_list.insert(13,"-")
    print("-----------------------------")
    print(inner_list)
    print("-----------------------------")
    
    #整理成直的資料
    count = 1
    num_year = 1
    for a in inner_list:
        try:
            if a == "-" or float(a):  #當月沒資料(-)或是有資料
                if count == 13: #超過12月的時候，再從1月開始計算
                    count = 1
                    num_year += 14
                data["作物"].append(inner_list[0])
                data["月份"].append(inner_list[num_year][:-1]+"/"+str(count))
                data["價格"].append(a)
                count += 1
        except:
            continue
    df = pd.DataFrame(data) 
    print("已抓取%s的資料" % (inner_list[0]))
    print(df)
    print("------------------------------------------")
    time.sleep(20)

<ipython-input-217-75bce5de5238>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  select = Select(driver.find_element_by_name("CropName"))
<ipython-input-217-75bce5de5238>:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_id("btn-search")
<ipython-input-217-75bce5de5238>:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  inner_content = driver.find_elements_by_css_selector("tbody>tr>td")


['大豆(元/公斤)', '2020年', '-', '83', '-', '-', '66.67', '50.52', '50', '-', '-', '-', '-', '-', '大豆(元/公斤)', '2019年', '-', '-', '-', '-', '67', '61.20', '52.77', '-', '-', '-', '62', '70.67']
-----------------------------
['大豆(元/公斤)', '2020年', '-', '83', '-', '-', '66.67', '50.52', '50', '-', '-', '-', '-', '-', '大豆(元/公斤)', '2019年', '-', '-', '-', '-', '67', '61.20', '52.77', '-', '-', '-', '62', '70.67']
-----------------------------
已抓取大豆(元/公斤)的資料
          作物       月份     價格
0   大豆(元/公斤)   2020/1      -
1   大豆(元/公斤)   2020/2     83
2   大豆(元/公斤)   2020/3      -
3   大豆(元/公斤)   2020/4      -
4   大豆(元/公斤)   2020/5  66.67
5   大豆(元/公斤)   2020/6  50.52
6   大豆(元/公斤)   2020/7     50
7   大豆(元/公斤)   2020/8      -
8   大豆(元/公斤)   2020/9      -
9   大豆(元/公斤)  2020/10      -
10  大豆(元/公斤)  2020/11      -
11  大豆(元/公斤)  2020/12      -
12  大豆(元/公斤)   2019/1      -
13  大豆(元/公斤)   2019/2      -
14  大豆(元/公斤)   2019/3      -
15  大豆(元/公斤)   2019/4      -
16  大豆(元/公斤)   2019/5     67
17  大豆(元/公斤)   2019/6  61.20


<ipython-input-217-75bce5de5238>:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  inner_content = driver.find_elements_by_css_selector("tbody>tr>td")


['大豆(黑豆)(元/公斤)', '2022年', '83', '83', '-', '大豆(黑豆)(元/公斤)', '2021年', '83', '83', '-', '-', '83', '50.83', '50', '-', '-', '-', '49.17', '53.67']
-----------------------------
['大豆(黑豆)(元/公斤)', '2021年', '83', '83', '-', '-', '83', '50.83', '50', '-', '-', '-', '49.17', '53.67']
-----------------------------
已抓取大豆(黑豆)(元/公斤)的資料
              作物       月份     價格
0       大豆(元/公斤)   2020/1      -
1       大豆(元/公斤)   2020/2     83
2       大豆(元/公斤)   2020/3      -
3       大豆(元/公斤)   2020/4      -
4       大豆(元/公斤)   2020/5  66.67
5       大豆(元/公斤)   2020/6  50.52
6       大豆(元/公斤)   2020/7     50
7       大豆(元/公斤)   2020/8      -
8       大豆(元/公斤)   2020/9      -
9       大豆(元/公斤)  2020/10      -
10      大豆(元/公斤)  2020/11      -
11      大豆(元/公斤)  2020/12      -
12      大豆(元/公斤)   2019/1      -
13      大豆(元/公斤)   2019/2      -
14      大豆(元/公斤)   2019/3      -
15      大豆(元/公斤)   2019/4      -
16      大豆(元/公斤)   2019/5     67
17      大豆(元/公斤)   2019/6  61.20
18      大豆(元/公斤)   2019/7  52.77
19      大豆(元/公斤

<ipython-input-217-75bce5de5238>:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  inner_content = driver.find_elements_by_css_selector("tbody>tr>td")


...

['轎篙筍(去殼生筍)(元/公斤)', '2021年', '-', '-', '-', '-', '40.83', '55', '-', '-', '-', '-', '-', '-', '轎篙筍(去殼生筍)(元/公斤)', '2020年', '-', '-', '-', '-', '26.67', '27.50', '-', '-', '-', '-', '54.5', '-', '轎篙筍(去殼生筍)(元/公斤)', '2019年', '-', '-', '-', '-', '31.50', '38', '-', '-', '-', '-', '-', '-']
-----------------------------
['轎篙筍(去殼生筍)(元/公斤)', '2021年', '-', '-', '-', '-', '40.83', '55', '-', '-', '-', '-', '-', '-', '轎篙筍(去殼生筍)(元/公斤)', '2020年', '-', '-', '-', '-', '26.67', '27.50', '-', '-', '-', '-', '54.5', '-', '轎篙筍(去殼生筍)(元/公斤)', '2019年', '-', '-', '-', '-', '31.50', '38', '-', '-', '-', '-', '-', '-']
-----------------------------
已抓取轎篙筍(去殼生筍)(元/公斤)的資料
                    作物       月份     價格
0             大豆(元/公斤)   2020/1      -
1             大豆(元/公斤)   2020/2     83
2             大豆(元/公斤)   2020/3      -
3             大豆(元/公斤)   2020/4      -
4             大豆(元/公斤)   2020/5  66.67
...                ...      ...    ...
31213  轎篙筍(去殼生筍)(元/公斤)   2019/8      -
31214  轎篙筍(去殼生筍)(元/公斤)   2019/9   

<ipython-input-217-75bce5de5238>:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  inner_content = driver.find_elements_by_css_selector("tbody>tr>td")


['鶯歌桃(元/公斤)', '2020年', '-', '-', '-', '-', '-', '130', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2019年', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2018年', '-', '-', '-', '-', '-', '44.75', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2017年', '-', '-', '-', '-', '-', '40', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2016年', '-', '-', '-', '-', '-', '39.70', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2015年', '-', '-', '-', '42.67', '43.67', '-', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2014年', '-', '-', '-', '55.83', '50.33', '-', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2013年', '-', '-', '-', '46.67', '49', '-', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2012年', '-', '-', '-', '48.33', '60.33', '-', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2011年', '-', '-', '-', '49.17', '47.67', '-', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2010年', '-', '-', '-', '53.75', '39.17', '-', '-', '-', '-', '-', '-', '-', '鶯歌桃(元/公斤)', '2009年', '-', '-', '-', '41'

<ipython-input-217-75bce5de5238>:14: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  inner_content = driver.find_elements_by_css_selector("tbody>tr>td")


['蘿蔔(元/公斤)', '2022年', '7.03', '8.17', '8.42', '蘿蔔(元/公斤)', '2021年', '12.4', '4.85', '5.8', '6.3', '9.35', '14.1', '5.1', '6.2', '-', '5.46', '8.09', '7.54', '蘿蔔(元/公斤)', '2020年', '8.34', '8.40', '7.11', '6.77', '7.55', '7.92', '3.53', '-', '11.87', '9.86', '23.33', '11.38', '蘿蔔(元/公斤)', '2019年', '5.35', '6.49', '5.90', '6.72', '8.43', '9.70', '10.77', '8.40', '5.13', '7.43', '5.94', '75', '蘿蔔(元/公斤)', '2018年', '8.10', '7.44', '5.68', '6.5', '7.08', '9.43', '-', '-', '16', '7.63', '4.68', '4.98', '蘿蔔(元/公斤)', '2017年', '5.59', '5.96', '6.85', '8', '9.18', '8.92', '-', '6.20', '5.82', '8.32', '13.3', '14.89', '蘿蔔(元/公斤)', '2016年', '6.44', '14.28', '16.77', '13.40', '11.87', '13.25', '14.64', '9.88', '10', '20.01', '20', '9.15', '蘿蔔(元/公斤)', '2015年', '11.11', '9.06', '8.59', '10.13', '12.03', '13.13', '13.50', '15.57', '20.17', '24', '24.83', '23.33', '蘿蔔(元/公斤)', '2014年', '11.53', '7.51', '7.41', '7.32', '8.92', '11.05', '13', '12.83', '13.33', '12.17', '11.50', '10.83', '蘿蔔(元/公斤)', '2013年', '15.

In [218]:
#建立EXCEL
# excel存放的資料夾
file_dir="/Users/ben/Documents/華梵課程"
# 若資料夾不存在，建立指定資料夾
file_out=file_dir+"/20220425_農產品.xlsx"
# 如果檔案不存在，建立檔案
if  os.path.exists(file_out) == False:
    df.to_excel(file_out,index=False)
    